<center> <h1> Convert Elastic Rules JSON to Playbook </h1> </center>

### A. Deskripsi

**Program ini berfungsi untuk men-convert file Elastic Rules JSON ke bentuk File Playbook Excel**

### B. Requirements

Berikut adalah requirements yang diperlukan untuk program ini : 
- python 3.12.3
- pandas 2.1.4
- tqdm 4.66.1
- httpx 0.27.0
- openpyxl 3.1.2

### C. Inisialisasi Awal

Berikut adalah yang harus dilakukan terlebih dahulu yaitu :
1. Export file Elastic SIEM dalam format NDJSON
2. Copy file tersebut ke program directory "files"
3. Ganti "ELASTIC_RULES_JSON_FILE_PATH" dengan file path
4. Isi ChatGPT API pada variabel "API_KEY"
5. Kurangi "BATCH_SIZE" untuk meningkatkan speed (Default = 10)

### D. Source Code

#### 1. Import library yang diperlukan

In [11]:
# Library untuk membaca file JSON
# Library untuk menyimpan file dalam format Excel
import pandas as pd
# Library untuk mengirim HTTP Request dengan performansi yang cepat
import httpx
# Library untuk membuat fungsi Asynchronous
import asyncio
# Library untuk membuat progress bar
from tqdm.notebook import trange

#### 2. Deklarasi Konstanta/Variabel Global

In [12]:
# Nama file elastic rules dalam format .ndjson
ELASTIC_RULES_JSON_FILE_PATH = "../files/[your-file]"
# List-list berisi nama kolom baru yang akan ditambahkan kepada dataframe yang baru
KOLOM_YANG_AKAN_DITAMBAHKAN = [
    "Created Date", "Name", "Description", "Impact", "Index", 
    "Query", "Tactic", "Tactic ID", "Technique", 
    "Technique ID", "References", "Tags"
]
# API URL untuk ChatGPT
API_URL = "https://api.openai.com/v1/chat/completions"
API_KEY = ""
# Delay dalam satuan detik
DELAY = 10
# Jumlah HTTP request yang dikirim dalam satu waktu
BATCH_SIZE = 10
# Nama file yang akan disimpan dalam bentuk file Excel diambil dari "ELASTIC_RULES_JSON_FILE_PATH"
NAMA_FILE_OUTPUT = ELASTIC_RULES_JSON_FILE_PATH.split('.')[-2].split('/')[-1]

#### 3. Membuat fungsi

In [13]:
# Fungsi untuk membaca file elastic rules dalam format .ndjson lalu membuat dataframe dari file tersebut
def membuat_dataframe_elastic(ELASTIC_RULES_JSON_FILE_PATH):
    
    dataframe_elastic = pd.read_json(ELASTIC_RULES_JSON_FILE_PATH, lines=True)
    return dataframe_elastic

In [14]:
# Fungsi untuk membuat dataframe baru yang akan disimpan dalam file berformat Excel
def membuat_dataframe_baru(KOLOM):
    
    # Membuat dataframe baru (kosong) yang berisikan kolom-kolom baru yang akan ditambahkan 
    dataframe_baru = pd.DataFrame(columns=KOLOM)
    return dataframe_baru

In [15]:
# Fungsi membuat list yang berisikan prompt-prompt yang dibagi menjadi beberapa jenis prompt
# Prompt Impact dan Prompt Description
def membuat_prompt_list(rule_name_list):
    
    # Membuat list kosong mengenai kolom yang akan diisi dengan menggunakan ChatGPT
    # Bisa menambahkan list kosong baru untuk kolom yang lain
    impact_prompt_list = []
    description_prompt_list = []

    for rule_name in rule_name_list:
        # Jika nama rule yang telah di-split lebih dari 2 string maka variabel diisi gabungan dari string kedua terakhir dan terakhir
        if( len(str(rule_name).split('-')) > 2 ):
            rule_name = str(rule_name).split('-')[-2] + ' ' + str(rule_name).split('-')[-1]
        else:
            # Jika nama rule yang telah di-split kurang dari 2 string maka variabel diisi dari terakhir
            rule_name = str(rule_name).split('-')[-1]

        impact_prompt = f"Please give me an impact of {rule_name} with maximal one sentence."
        description_prompt = f"Please give me a description of {rule_name} with maximal one sentence."

        # Mengisi prompt yang telah digabung dengan nama rule dan dimasukkan pada list kosong yang dibuat tadi
        impact_prompt_list.append(impact_prompt)
        description_prompt_list.append(description_prompt)
        
    return impact_prompt_list, description_prompt_list

In [16]:
# Fugsi untuk membuat kolom baru dengan bantuan ChatGPT untuk mengisi kolom tersebut
# Kolom "Description" dan kolom "Impact" diisi oleh bantuan ChatGPT
async def membuat_kolom_baru(BATCH_SIZE, impact_prompts_list, description_prompts_list):
    
    # Membuat list baru yang kosong yang akan menampung hasil dari prompt ChatGPT yang telah dikirim
    impact_hasil_prompt_list = []
    description_hasil_prompt_list = []
    
    # Mengirim ChatGPT request dengan batching
    for i in trange(0, len(impact_prompts_list), BATCH_SIZE, desc="Progress Mengirim Prompt"):
        # Membuat task untuk dijalankan secara asynchronous
        impact = [api_request(prompt) for prompt in impact_prompts_list[i:i+BATCH_SIZE]]
        description = [api_request(prompt) for prompt in description_prompts_list[i:i+BATCH_SIZE]]

        # Menjalankan task tersebut secara asynchronous
        response_impact = await asyncio.gather(*impact)
        response_description = await asyncio.gather(*description)

        # Menggabungkan list "response_impact" dan list "response_description" dengan list "impact_hasil_prompt_list" dan "description_hasil_prompt_list"
        impact_hasil_prompt_list += response_impact
        description_hasil_prompt_list += response_description

        print(f"Sleep {DELAY} detik...")

        # Delay sebanyak n detik
        await asyncio.sleep(DELAY)
    
    return impact_hasil_prompt_list, description_hasil_prompt_list

In [17]:
# Fungsi untuk mengirim ChatGPT API Request
async def api_request(prompt):
    headers = {
        "Authorization": f"Bearer {API_KEY}",
        "Content-Type": "application/json"
    }

    data = {
        "model": "gpt-4",
        "messages": [
            {"role": "system", "content": "You are a Cyber Secutiy Professional."},
            {"role": "user", "content": prompt}
        ]
    }
    async with httpx.AsyncClient() as client:
        print(f"Mengirim prompt {prompt}")
        response = await client.post(API_URL, json=data, headers=headers, timeout=None)
        # Mengirimkan status error jika terjadi error pada request
        response.raise_for_status()
        response_data = response.json()
        # Hanya mengirim berupa hasil prompt nya saja
        return response_data["choices"][0]["message"]["content"]

In [18]:
# Fungsi untuk mengisi data dari dataframe elastic ke dataframe baru dan mengisi data dari ChatGPT ke dataframe baru
def mengisi_data_ke_dataframe(dataframe_elastic, dataframe_baru, impact_hasil_prompt_list, description_hasil_prompt_list):
    # Looping sampai keseluruhan data pada dataframe elastic
    for i in trange(len(dataframe_elastic), desc="Progress Mengisi Data ke Dataframe Baru"):
        # Mengecek apakah isi dari kolom "data_view_id" bernilai "NULL" pada dataframe elastic
        # Jika iya, maka variabel "index" akan diisi dari kolom "index" dari dataframe elastic
        if(pd.isnull(dataframe_elastic["data_view_id"][i])):
            index = dataframe_elastic["index"][i]
        else:
            index = dataframe_elastic["data_view_id"][i]

        # Memotong string yang berupa timestamp dari kolom "created_at"
        # Mengambil string pada index pertama dikarenakan hanya berupa tanggal
        # Jika mengambil keseluruhan timestamp dan tidak dikonversi ke dalam format "string" maka ketika dilakukan proses konversi ke File Excel akan Error
        date = str(dataframe_elastic["created_at"][i]).split(' ')[0]

        # Mengisi kolom "Impact" dan kolom "Description"
        impact = impact_hasil_prompt_list[i]
        description = description_hasil_prompt_list[i]
        
        # Mengecek apakah terdapat isi dari kolom "threat" pada dataframe elastic
        # Tidak bisa pakai fungsi pd.isnull() dikarenakan akan dianggap tidak "NULL" sehingga mengecek isi dari kolom "threat" pada dataframe elastic memakai fungsi len()
        if(len(dataframe_elastic["threat"][i]) == 0):
            # Jika tidak ada isinya kolom "threat" pada dataframe elastic maka tactic, tactic ID, technique, dan technique ID pun tidak ada 
            tactic = '-'
            tactic_id = '-'
            technique = '-'
            technique_id = '-'
        else:
            # Jika ada isinya kolom "threat" pada dataframe elastic maka dilakukan pengecekkan pada key "tactic"
            # Tidak bisa pakai fungsi pd.isnull() dikarenakan akan dianggap tidak "NULL"
            if(len(dataframe_elastic["threat"][i][0]["tactic"]) == 0):
                # Jika tidak ada isinya pada key "tactic" maka tactic, tactic ID, technique, dan technique ID pun tidak ada 
                tactic = '-'
                tactic_id = '-'
                technique = '-'
                technique_id = '-'
            else:
                # Jika ada isinya kolom "threat" pada dataframe elastic dan key "tactic" maka akan diassign ke variabel "tactic" dan "tactic_id"
                # Harus diconvert ke dalam tipe "string"
                tactic = str(dataframe_elastic["threat"][i][0]["tactic"]["name"])
                tactic_id = str(dataframe_elastic["threat"][i][0]["tactic"]["id"])

                # Jika ada isinya kolom "threat" pada dataframe elastic dilakukan pengecekkan pada key "technique"
                if(len(dataframe_elastic["threat"][i][0]["technique"]) == 0):
                    technique = '-'
                    technique_id = '-'
                else:
                    # Jika ada isinya kolom "threat" pada dataframe elastic dan key "technique" maka akan diassign ke variabel "technique" dan "technique_id"
                    # Harus diconvert ke dalam tipe "string"
                    technique = str(dataframe_elastic["threat"][i][0]["technique"][0]["name"])
                    technique_id = str(dataframe_elastic["threat"][i][0]["technique"][0]["id"])

        
            # Mengisi data ke dataframe baru
            # Pada pandas versi ini maka harus dilakukan assign kolom dan baris dengan fungsi df.loc[i] = []
            dataframe_baru.loc[i] = [
                date, dataframe_elastic["name"][i], description, impact, str(index), dataframe_elastic["query"][i],
                tactic, tactic_id, technique, technique_id,
                str(dataframe_elastic["references"][i]), str(dataframe_elastic["tags"][i])
            ]

    return dataframe_baru

#### 4. Membuat fungsi utama

In [19]:
async def main():

    # Menjalankan fungsi membuat_dataframe_elastic()
    dataframe_elastic = membuat_dataframe_elastic(ELASTIC_RULES_JSON_FILE_PATH=ELASTIC_RULES_JSON_FILE_PATH)

    # Menjalankan fungsi membuat_prompt_list()
    impact_prompts_list, description_prompts_list = membuat_prompt_list(rule_name_list=dataframe_elastic["name"])

    # Menjalankan fungsi membuat_kolom_baru() secara Asynchronous
    impact_hasil_prompt_list, description_hasil_prompt_list = await membuat_kolom_baru(BATCH_SIZE, impact_prompts_list, description_prompts_list)


    # Membuat dataframe baru berdasarkan kolom-kolom yang dibutuhkan
    dataframe_baru = membuat_dataframe_baru(KOLOM=KOLOM_YANG_AKAN_DITAMBAHKAN)

    # Menjalankan fungsi mengisi_data_ke_dataframe()
    dataframe_baru = mengisi_data_ke_dataframe(dataframe_elastic, dataframe_baru, impact_hasil_prompt_list, description_hasil_prompt_list)

    print(f"Menyimpan file {NAMA_FILE_OUTPUT}.xlsx")
    dataframe_baru.to_excel("../files/" + NAMA_FILE_OUTPUT + ".xlsx", index=False)

#### 5. Menjalankan program

In [20]:
# Menjalankan fungsi main()
await main()

Progress Mengirim Prompt:   0%|          | 0/5 [00:00<?, ?it/s]

Mengirim prompt Please give me an impact of  Execution Suspicious msi Installer executed with maximal one sentence.
Mengirim prompt Please give me an impact of Scheduled Task Created or Updated Success with maximal one sentence.
Mengirim prompt Please give me an impact of  Exfiltration   New External Disk/Volume Detected with maximal one sentence.
Mengirim prompt Please give me an impact of  "Impacket" Tools Detected (Python based) with maximal one sentence.
Mengirim prompt Please give me an impact of  Lockbit 3.0 with maximal one sentence.
Mengirim prompt Please give me an impact of  Possible Powershell .ps1 Script Use Over SMB with maximal one sentence.
Mengirim prompt Please give me an impact of  File or Directory Deletion Command with maximal one sentence.
Mengirim prompt Please give me an impact of  Suspicious Link in Message Created matches Threat Intelligence with maximal one sentence.
Mengirim prompt Please give me an impact of  Outbound Connection from IoC Cuba Ransomware   IP

Progress Mengisi Data ke Dataframe Baru:   0%|          | 0/50 [00:00<?, ?it/s]

Menyimpan file rules_BSI - Copy.xlsx
